In [10]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import LSTM
from dictionaries import *

BORDER = "==============================================================="

In [2]:
def load_data(filename):
    raw_text = open(filename).read()
    text_list = raw_text.strip().split('\n')
    return text_list

In [4]:
def remove_int(text):
    '''
    Remove numbers from strings
    '''
    new_text = []
    for line in text:
        # Remove integer values
        no_digits = ''.join([i for i in line if not i.isdigit()])

        # Remove punctuation
        new_text.append(no_digits)
    return new_text


def remove_empty(text):
    '''
    Removes all empty string
    '''
    new_text = []
    for line in text:
        if not line.isspace():
            new_text.append(line)
    return new_text


def lowercase(text):
    '''
    Convert text to all lowercase and remove punctuation and numbers
    '''
    new_text = []
    for line in text:
        # Make text lowercase
        line = line.lower()

        new_text.append(line)
    return new_text


def remove_punctuation(text):
    '''
    Convert text to all lowercase and remove punctuation and numbers
    '''
    new_text = []
    for line in text:
        sentence = ''.join([i for i in line])

        # Remove punctuation
        new_text.append(sentence.translate(
            str.maketrans('', '', string.punctuation)))
    return new_text

def separate_sonnets(text):
    return text.split('\n\n\n')

In [17]:
def process_data_RNN(text_list, verbose=0):
    '''
    Create fixed length training sequences of length 40 char from the sonnet
    corpus (REVERSED).

    Input: Text file in the form of list of words

    Output: X, Y, dataX, dataY, int_to_char, n_vocab
    '''

    print("Processing datafile....")

    # Preprocessing
    text_list = remove_int(text_list)
    text_list = remove_empty(text_list)
    text_list = lowercase(text_list)

    new_text = '\n'.join(text_list)
    
    # Separate into sonnets
    sonnets = separate_sonnets(new_text)
    for i, sonnet in enumerate(sonnets):
        sonnets[i] = sonnet[::-1]

    if verbose == 1:
        print(BORDER)
        print("Processed text")
        for i, sonnet in enumerate(sonnets):
            print(BORDER)
            print("Sonnet ", i, ": ")
            print(sonnet)
            print(BORDER)
        print(new_text)
        print(BORDER)

    # create mapping of unique chars to integers, and a reverse mapping
    chars = sorted(list(set(new_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # summarize the loaded data
    n_chars = len(new_text)
    n_vocab = len(chars)
    n_sonnets = len(sonnets)

    # prepare the dataset of input to output pairs encoded as integers
    seq_length = 40
    dataX = []
    dataY = []
    sonnetInd = []  # list of indices that denotes the start
    dataStart = []
    for sonnet in sonnets:
        for i in range(0, len(sonnet) - seq_length):
            seq_in = sonnet[i:i + seq_length]
            seq_out = sonnet[i + seq_length]
            dataX.append([char_to_int[char] for char in seq_in])
            dataY.append(char_to_int[seq_out])

            if i == 0:
                sonnetInd.append(len(dataX) - 1)
                dataStart.append([char_to_int[char] for char in seq_in])
    n_patterns = len(dataX)

    if verbose == 1:
        print(BORDER)
        print("Processed Text Summary")
        print("Total Characters: ", n_chars)
        print("Total Vocab: ", n_vocab)
        print("Total Patterns: ", n_patterns)
        print("Number of Sonnets: ", n_sonnets)
        print(BORDER)

    X_start = np.zeros((n_sonnets, seq_length, n_vocab))
    for j, i in enumerate(sonnetInd):
        sentence = dataX[i]
        for t, ind in enumerate(sentence):
            X_start[j, t, ind] = 1

    X = np.zeros((n_patterns, seq_length, n_vocab))
    y = np.zeros((n_patterns, n_vocab))
    for i, sentence in enumerate(dataX):
        for t, ind in enumerate(sentence):
            X[i, t, ind] = 1
        y[i, dataY[i]] = 1

    return X, y, dataX, dataY, dataStart, int_to_char, char_to_int


In [16]:
def train_LSTM(X, y, verbose=0):
    '''
    Takes training data X and Y and returns the fitted LSTM model

    Input:
        X : a list of sequences of int
        Y : one-hot encoding of the int coming after the sequence
    '''

    print("Building Model...")

    # Take a subset of sequences
    X = X[0::5]
    y = y[0::5]

    # define the LSTM model
    model = Sequential()
    model.add(LSTM(180, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1], activation='softmax'))

    optimizer = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])

    # fit the model
    model.fit(X, y, epochs=10, batch_size=512, verbose=verbose)
    return model


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
def generate_text(model, dataStart, int_to_char, char_to_int,
                  diversity=1.0, seed=0, verbose=0):
    '''
    Given model, dataX, int_to_char, n_vocab returns generated_text using
    predict function

    Input:
        model: the LSTM model that we trained
        dataX: list of sequences
        int_to_char: a dictionary matching interger to specific character
        n_vocab: number of unique characters we have

    Output: generate_text as string

    '''

    print("Generating text...")

    n_vocab = len(int_to_char)
    size = len(dataStart[0])

    if seed == 0:
        # pick a random seed
        start = np.random.randint(0, len(dataStart) - 1)
        pattern = dataStart[start]

    else:
        seed = 'shall i compare thee to a summer\'s day?\n'
        seed = seed[::-1]
        pattern = [char_to_int[char] for char in seed]

    seq = [int_to_char[value] for value in pattern]

    if verbose == 1:
        print("Seed: ", ''.join(seq))

    # generate characters
    num_lines = 1
    num_char = 0
    max_num_lines = 14
    max_char = 400
    while num_lines < max_num_lines and num_char < max_char:
        # Create and normalize x to be input of RNN
        x = np.zeros((1, size, n_vocab))
        for t, char in enumerate(pattern):
            x[0, t, char] = 1

        # Make prediction using trained model
        prediction = model.predict(x, verbose=verbose)[0]
        index = sample(prediction, diversity)

        # Convert prediction to character
        result = int_to_char[index]

        # Add prediction to pattern and set to size 40
        pattern.append(index)
        pattern = pattern[1:1 + size]

        # Add result to seq
        seq.append(result)
        
        num_char += 1

        if verbose == 1:
            print(BORDER)
            print("selected char: ", result)
            print("new pattern: ",
                  ''.join([int_to_char[value] for value in pattern]))
            print(BORDER)

    # Return seq as string
    return ''.join(seq)

In [9]:
def save_textfile(filename, text):
    '''
    Given filename and text, save text in file

    Input: filename and text as string
    '''
    print("Saving generated text...")
    f = open(filename, 'w')
    f.write(text)
    f.close()
    return 0

In [18]:
file = 'data/shakespeare.txt'

verbose = 1

text_list = load_data(file)
(X, y, dataX, dataY, dataStart, int_to_char, char_to_int) = (process_data_RNN(text_list, verbose=verbose))

Processing datafile....
Processed text
Sonnet  0 : 
.eeht dna evarg eht yb ,eud s'dlrow eht tae ot  
,eb nottulg siht esle ro ,dlrow eht ytip  
:gnidraggin ni etsaw ts'kam lruhc rednet dna
,tnetnoc yht tseirub dub nwo eniht nihtiw
,gnirps yduag eht ot dlareh ylno dna
,tnemanro hserf s'dlrow eht won tra taht uoht
:leurc oot fles teews yht ot ,eof yht fles yht
,seil ecnadnuba erehw enimaf a gnikam
,leuf laitnatsbus-fles htiw emalf s'thgil yht ts'deef
,seye thgirb nwo eniht ot detcartnoc uoht tub
:yromem sih raeb thgim rieh rednet sih
,esaeced emit yb dluohs repir eht sa tub
,eid reven thgim esor s'ytuaeb ybereht taht
,esaercni erised ew serutaerc tseriaf morf

Sonnet  1 : 
.dloc ti ts'leef uoht nehw mraw doolb yht ees dna  
,dlo tra uoht nehw edam wen eb ot erew siht  
.eniht noisseccus yb ytuaeb sih gnivorp
'esucxe dlo ym ekam dna ,tnuoc ym mus llahs
enim fo dlihc riaf siht' rewsna tsdluoc uoht fi
,esu s'ytuaeb yht devresed esiarp erom hcum woh
.esiarp sseltfirht dna ,emahs gnitae-lla n

Processed Text Summary
Total Characters:  94596
Total Vocab:  38
Total Patterns:  87977
Number of Sonnets:  154


In [20]:
model = train_LSTM(X, y, verbose=verbose)

Building Model...
Epoch 1/10
17596/17596 [==============================] - 37s 2ms/step - loss: 3.1681 - acc: 0.1530
Epoch 2/10
17596/17596 [==============================] - 35s 2ms/step - loss: 2.9644 - acc: 0.1664
Epoch 3/10
17596/17596 [==============================] - 34s 2ms/step - loss: 2.9187 - acc: 0.1691
Epoch 4/10
17596/17596 [==============================] - 35s 2ms/step - loss: 2.8360 - acc: 0.1985
Epoch 5/10
17596/17596 [==============================] - 33s 2ms/step - loss: 2.6998 - acc: 0.2505
Epoch 6/10
17596/17596 [==============================] - 34s 2ms/step - loss: 2.5629 - acc: 0.2985
Epoch 7/10
17596/17596 [==============================] - 33s 2ms/step - loss: 2.4645 - acc: 0.3210
Epoch 8/10
17596/17596 [==============================] - 33s 2ms/step - loss: 2.3955 - acc: 0.3305
Epoch 9/10
17596/17596 [==============================] - 34s 2ms/step - loss: 2.3356 - acc: 0.3389
Epoch 10/10
17596/17596 [==============================] - 34s 2ms/step - loss: 2.

In [ ]:
verbose = 1
seed = 1
diversity = 0.5
generated = generate_text(model, dataStart, int_to_char, char_to_int, diversity=diversity, seed=seed, verbose=verbose)
print(generated[::-1])

Generating text...
Seed:  
?yad s'remmus a ot eeht erapmoc i llahs
1/1 [==============================] - 0s 8ms/step
selected char:   
new pattern:  ?yad s'remmus a ot eeht erapmoc i llahs 
1/1 [==============================] - 0s 9ms/step
selected char:  r
new pattern:  yad s'remmus a ot eeht erapmoc i llahs r
1/1 [==============================] - 0s 7ms/step
selected char:  o
new pattern:  ad s'remmus a ot eeht erapmoc i llahs ro
1/1 [==============================] - 0s 9ms/step
selected char:  o
new pattern:  d s'remmus a ot eeht erapmoc i llahs roo
1/1 [==============================] - 0s 10ms/step
selected char:   
new pattern:   s'remmus a ot eeht erapmoc i llahs roo 
1/1 [==============================] - 0s 13ms/step
selected char:  r
new pattern:  s'remmus a ot eeht erapmoc i llahs roo r
1/1 [==============================] - 0s 7ms/step
selected char:  o
new pattern:  'remmus a ot eeht erapmoc i llahs roo ro
1/1 [==============================] - 0s 13ms/step
selected ch

selected char:  t
new pattern:   llahs roo ro yt
,dera sero emiht eniw t
1/1 [==============================] - 0s 9ms/step
selected char:  a
new pattern:  llahs roo ro yt
,dera sero emiht eniw ta
1/1 [==============================] - 0s 7ms/step
selected char:  h
new pattern:  lahs roo ro yt
,dera sero emiht eniw tah
1/1 [==============================] - 0s 9ms/step
selected char:  t
new pattern:  ahs roo ro yt
,dera sero emiht eniw taht
1/1 [==============================] - 0s 8ms/step
selected char:   
new pattern:  hs roo ro yt
,dera sero emiht eniw taht 
1/1 [==============================] - 0s 7ms/step
selected char:  e
new pattern:  s roo ro yt
,dera sero emiht eniw taht e
1/1 [==============================] - 0s 6ms/step
selected char:  t
new pattern:   roo ro yt
,dera sero emiht eniw taht et
1/1 [==============================] - 0s 7ms/step
selected char:  a
new pattern:  roo ro yt
,dera sero emiht eniw taht eta
1/1 [==============================] - 0s 7ms/step
selected

1/1 [==============================] - 0s 19ms/step
selected char:  e
new pattern:   eniw taht eta evi dna seht ym dna sim e
1/1 [==============================] - 0s 16ms/step
selected char:  b
new pattern:  eniw taht eta evi dna seht ym dna sim eb
1/1 [==============================] - 0s 7ms/step
selected char:   
new pattern:  niw taht eta evi dna seht ym dna sim eb 
1/1 [==============================] - 0s 6ms/step
selected char:  e
new pattern:  iw taht eta evi dna seht ym dna sim eb e
1/1 [==============================] - 0s 7ms/step
selected char:  t
new pattern:  w taht eta evi dna seht ym dna sim eb et
1/1 [==============================] - 0s 8ms/step
selected char:   
new pattern:   taht eta evi dna seht ym dna sim eb et 
1/1 [==============================] - 0s 6ms/step
selected char:  e
new pattern:  taht eta evi dna seht ym dna sim eb et e
1/1 [==============================] - 0s 7ms/step
selected char:  m
new pattern:  aht eta evi dna seht ym dna sim eb et em
1/1 [=

selected char:   
new pattern:  a sim eb et em eht eroh eb tni seht yot 
1/1 [==============================] - 0s 8ms/step
selected char:  y
new pattern:   sim eb et em eht eroh eb tni seht yot y
1/1 [==============================] - 0s 6ms/step
selected char:  e
new pattern:  sim eb et em eht eroh eb tni seht yot ye
1/1 [==============================] - 0s 9ms/step
selected char:  h
new pattern:  im eb et em eht eroh eb tni seht yot yeh
1/1 [==============================] - 0s 7ms/step
selected char:  t
new pattern:  m eb et em eht eroh eb tni seht yot yeht
1/1 [==============================] - 0s 7ms/step
selected char:   
new pattern:   eb et em eht eroh eb tni seht yot yeht 
1/1 [==============================] - 0s 10ms/step
selected char:  e
new pattern:  eb et em eht eroh eb tni seht yot yeht e
1/1 [==============================] - 0s 6ms/step
selected char:  h
new pattern:  b et em eht eroh eb tni seht yot yeht eh
1/1 [==============================] - 0s 11ms/step
select

1/1 [==============================] - 0s 8ms/step
selected char:  i
new pattern:  ht yot yeht eht ym dneht ot dna siht hti
1/1 [==============================] - 0s 8ms/step
selected char:  r
new pattern:  t yot yeht eht ym dneht ot dna siht htir
1/1 [==============================] - 0s 7ms/step
selected char:  w
new pattern:   yot yeht eht ym dneht ot dna siht htirw
1/1 [==============================] - 0s 9ms/step
selected char:   
new pattern:  yot yeht eht ym dneht ot dna siht htirw 
1/1 [==============================] - 0s 11ms/step
selected char:  e
new pattern:  ot yeht eht ym dneht ot dna siht htirw e
1/1 [==============================] - 0s 8ms/step
selected char:  g
new pattern:  t yeht eht ym dneht ot dna siht htirw eg
1/1 [==============================] - 0s 9ms/step
selected char:  i
new pattern:   yeht eht ym dneht ot dna siht htirw egi
1/1 [==============================] - 0s 7ms/step
selected char:  o
new pattern:  yeht eht ym dneht ot dna siht htirw egio
1/1 [==

...

selected char:   
new pattern:  dnaef revem tsiw yr
,evgno sniw erol ro 
1/1 [==============================] - 0s 6ms/step
selected char:  s
new pattern:  naef revem tsiw yr
,evgno sniw erol ro s
1/1 [==============================] - 0s 7ms/step
selected char:  n
new pattern:  aef revem tsiw yr
,evgno sniw erol ro sn
1/1 [==============================] - 0s 6ms/step
selected char:  i
new pattern:  ef revem tsiw yr
,evgno sniw erol ro sni
1/1 [==============================] - 0s 7ms/step
selected char:   
new pattern:  f revem tsiw yr
,evgno sniw erol ro sni 
1/1 [==============================] - 0s 8ms/step
selected char:  e
new pattern:   revem tsiw yr
,evgno sniw erol ro sni e
1/1 [==============================] - 0s 8ms/step
selected char:  v
new pattern:  revem tsiw yr
,evgno sniw erol ro sni ev
1/1 [==============================] - 0s 7ms/step
selected char:  o
new pattern:  evem tsiw yr
,evgno sniw erol ro sni evo
1/1 [==============================] - 0s 8ms/step
selected

selected char:  ,
new pattern:  rol ro sni evos ruoe  ni da
,eni sevo 
,
1/1 [==============================] - 0s 8ms/step
selected char:  e
new pattern:  ol ro sni evos ruoe  ni da
,eni sevo 
,e
1/1 [==============================] - 0s 7ms/step
selected char:  r
new pattern:  l ro sni evos ruoe  ni da
,eni sevo 
,er
1/1 [==============================] - 0s 8ms/step
selected char:  o
new pattern:   ro sni evos ruoe  ni da
,eni sevo 
,ero
1/1 [==============================] - 0s 7ms/step
selected char:  i
new pattern:  ro sni evos ruoe  ni da
,eni sevo 
,eroi
1/1 [==============================] - 0s 6ms/step
selected char:   
new pattern:  o sni evos ruoe  ni da
,eni sevo 
,eroi 
1/1 [==============================] - 0s 8ms/step
selected char:  t
new pattern:   sni evos ruoe  ni da
,eni sevo 
,eroi t
1/1 [==============================] - 0s 7ms/step
selected char:  s
new pattern:  sni evos ruoe  ni da
,eni sevo 
,eroi ts
1/1 [==============================] - 0s 6ms/step
selected

selected char:  i
new pattern:  sevo 
,eroi tseht eht roo  eht  niw
,eti
1/1 [==============================] - 0s 8ms/step
selected char:  h
new pattern:  evo 
,eroi tseht eht roo  eht  niw
,etih
1/1 [==============================] - 0s 6ms/step
selected char:  t
new pattern:  vo 
,eroi tseht eht roo  eht  niw
,etiht
1/1 [==============================] - 0s 9ms/step
selected char:   
new pattern:  o 
,eroi tseht eht roo  eht  niw
,etiht 
1/1 [==============================] - 0s 8ms/step
selected char:  r
new pattern:   
,eroi tseht eht roo  eht  niw
,etiht r
1/1 [==============================] - 0s 7ms/step
selected char:  o
new pattern:  
,eroi tseht eht roo  eht  niw
,etiht ro
1/1 [==============================] - 0s 8ms/step
selected char:  n
new pattern:  ,eroi tseht eht roo  eht  niw
,etiht ron
1/1 [==============================] - 0s 7ms/step
selected char:   
new pattern:  eroi tseht eht roo  eht  niw
,etiht ron 
1/1 [==============================] - 0s 7ms/step
selected

selected char:  e
new pattern:  iw
,etiht ron eht tsim at dna setiht ere
1/1 [==============================] - 0s 6ms/step
selected char:   
new pattern:  w
,etiht ron eht tsim at dna setiht ere 
1/1 [==============================] - 0s 10ms/step
selected char:  r
new pattern:  
,etiht ron eht tsim at dna setiht ere r
1/1 [==============================] - 0s 8ms/step
selected char:  a
new pattern:  ,etiht ron eht tsim at dna setiht ere ra
1/1 [==============================] - 0s 8ms/step
selected char:   
new pattern:  etiht ron eht tsim at dna setiht ere ra 
1/1 [==============================] - 0s 8ms/step
selected char:  t
new pattern:  tiht ron eht tsim at dna setiht ere ra t
1/1 [==============================] - 0s 7ms/step
selected char:  e
new pattern:  iht ron eht tsim at dna setiht ere ra te
1/1 [==============================] - 0s 7ms/step
selected char:  v
new pattern:  ht ron eht tsim at dna setiht ere ra tev
1/1 [==============================] - 0s 21ms/step
select

1/1 [==============================] - 0s 8ms/step
selected char:  n
new pattern:  iht ere ra tevir ero 
,evol seht tgnow n
1/1 [==============================] - 0s 8ms/step

In [ ]:
save = 'generated/shakespeare_seeded_1.5.txt'
save_textfile(save, generated)